![@mikegchambers](../../images/header.png)

# SageMaker SDK - Training a model in 'script mode' and deploying the endpoint. 

In this notebook we will look at how models can be trained, saved, loaded and run.  Then we will see how the same thing can be achived using the SageMaker SDK, and SageMaker managed infrastructure. 

This notebook represents a massively over engineered excercise, however the principles, if not the code itself, and be leveraged for much bigger projects.

# Create a dataset and save

In [1]:
from sklearn import datasets
import pickle

In [2]:
X, y = datasets.make_regression(100, 1, noise=5, bias=0)

In [4]:
pickle.dump([X,y], open('./train.pickle', 'wb'))

# Create a model from the dataset

In [5]:
from sklearn.linear_model import LinearRegression
import pickle

In [6]:
[XX, yy] = pickle.load(open('./train.pickle', 'rb'))

In [7]:
model = LinearRegression()

In [8]:
model.fit(XX,yy)

LinearRegression()

## Make a test prediction

In [9]:
model.predict([[0],[1],[2],[3]])

array([  0.71743699,  83.56792364, 166.41841029, 249.26889694])

# Save the model to a file

In [10]:
p = pickle.dumps(model)

In [11]:
pickle.dump(model, open('./model.pickle', 'wb'))

# Later load the model from a file

In [12]:
from sklearn.linear_model import LinearRegression
import pickle

In [13]:
loaded_model = pickle.load(open('./model.pickle', 'rb'))

## Make a test prediction

In [14]:
loaded_model.predict([[0],[1],[2],[3]])

array([  0.71743699,  83.56792364, 166.41841029, 249.26889694])

# SageMaker Training

In [15]:
import sagemaker
from sagemaker.sklearn.estimator import SKLearn
import boto3
import os

Create some variables that will be used through this process:

In [16]:
role = sagemaker.get_execution_role()
sess = sagemaker.Session()
bucket = sess.default_bucket()

s3_prefix = "script-mode-workflow"
pickle_s3_prefix = f"{s3_prefix}/pickle"
pickle_s3_uri = f"s3://{bucket}/{s3_prefix}/pickle"
pickle_train_s3_uri = f"{pickle_s3_uri}/train"

train_dir = os.path.join(os.getcwd(), "")

Upload the training data to S3, so it's available for SageMaker training:

In [17]:
s3_resource_bucket = boto3.Session().resource("s3").Bucket(bucket)
s3_resource_bucket.Object(os.path.join(pickle_s3_prefix, "train.pickle")).upload_file(
    train_dir + "/train.pickle"
)

Create some hyperparameters:

In [18]:
# This is not required as these values are the defaults:

hyperparameters = {
    "copy_X": True,
    "fit_intercept": True,
    "normalize": False,
}

More configuration for the model:

In [19]:
train_instance_type = "ml.m5.large"

inputs = {
    "train": pickle_train_s3_uri
}

The SageMaker Estimator object is a high level interface for SageMaker training.  This object represents the algorithm, the data, and other configuration. 

https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html

In [20]:
estimator_parameters = {
    "entry_point": "script.py",
    "source_dir": "script",
    "framework_version": "0.23-1",
    "py_version": "py3",
    "instance_type": train_instance_type,
    "instance_count": 1,
    "hyperparameters": hyperparameters,
    "role": role,
    "base_job_name": "linearregression-model",
}

estimator = SKLearn(**estimator_parameters)

When we call 'fit' SageMaker will spin up managed containers, transfer the code and data to the container and then start the training.  All this happens off of the notebook server.  We can watch the training through the console, and watch the logs in CloudWatch Logs.

In [21]:
estimator.fit(inputs)

2021-11-21 10:36:33 Starting - Starting the training job...
2021-11-21 10:36:58 Starting - Launching requested ML instancesProfilerReport-1637490993: InProgress
......
2021-11-21 10:37:59 Starting - Preparing the instances for training.........
2021-11-21 10:39:27 Downloading - Downloading input data
2021-11-21 10:39:27 Training - Downloading the training image...
2021-11-21 10:40:05 Uploading - Uploading generated training model.2021-11-21 10:39:59,591 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2021-11-21 10:39:59,594 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-11-21 10:39:59,604 sagemaker_sklearn_container.training INFO     Invoking user training script.
2021-11-21 10:39:59,845 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-11-21 10:39:59,857 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-11-21 10:39:59,872 sagemaker-

# SageMaker Endpoint

We can now take create a 'predictor' by deploying the estimator.  Then we can use it to make new predictions.

(Make sure that the 'endpoint_name' used is not currently running.)

In [23]:
sklearn_predictor = estimator.deploy(initial_instance_count=1,
                                     instance_type='ml.m5.large',
                                     endpoint_name='linearregression-endpoint')

-------!

In [24]:
sklearn_predictor.predict([[0],[1],[2],[3]])

array([  0.71743699,  83.56792364, 166.41841029, 249.26889694])

## Clean up

Running this cell will remove the endpoint and configuration:

In [ ]:
sklearn_predictor.delete_endpoint(True)